In [ ]:
!python -m spacy download de_core_news_lg
!pip install spacy

2023-12-06 14:33:53.916057: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 14:33:53.916146: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 14:33:53.916213: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 14:33:55.471097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_lg-3.6.0/de_core_news_lg-3.6.0-py3-none-any.whl (567.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('de_core

In [ ]:
import spacy  # version 3.0.6'
import hashlib
import re
import csv

#load language modell
nlp = spacy.load("de_core_news_lg")

#open list of public personas
file = open("public_figures_5.csv", "r")
public_figures = list(csv.reader(file, delimiter=","))[0]
public_figures_pattern = [{"label": "PER_PUBLIC", "pattern": name} for name in public_figures]

#could be a list of tiktok names
user_patterns = [{"label": "PER_USER", "pattern": "@tiktok_nickname"}]


ruler = nlp.add_pipe("entity_ruler", before='ner')
ruler.add_patterns(public_figures_pattern)
ruler.add_patterns(user_patterns)

#set SALT for encription
SALT ='asdt@'

In [ ]:
def anno(text:str, public_figures_pattern: list, user_patterns:list, salt: str, nlp) -> str:
  #filter IBAN
  iban_pattern = "[A-Z]{2}\d{2} ?\d{4} ?\d{4} ?\d{4} ?\d{4} ?[\d]{0,2}"
  text  = re.sub(iban_pattern,'ANNO@IBAN', text)

  #filter Email
  text = re.sub(r'[\w.+-]+@[\w-]+\.[\w.-]+','ANNO@MAIL',text)

  #filter phone numbers
  phone_pattern = '((?:\+\d{2}[-\.\s]??|\d{4}[-\.\s]??)?(?:\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}))'
  text = re.sub(phone_pattern,'ANNO@PHONE',text)

  doc = nlp(text)
  for ent in doc.ents:
    #hash people and tiktok names
    if ent.label_ == 'PER' or ent.label_ == 'PER_USER':
      text_salt = ent.text + SALT
      text = text.replace(ent.text, 'PerHash_' +str(int(hashlib.md5(text_salt.encode('utf-8')).hexdigest(), 16)))
    #Hash locations
    if ent.label_ == 'LOC':
      text_salt = ent.text + SALT
      text = text.replace(ent.text, 'LocHash_' +str(int(hashlib.md5(text_salt.encode('utf-8')).hexdigest(), 16)))
  return text




#Test sentences
test_names = "@tiktok_nickname und Tino sind nicht bekannt aber gute Kumpels. Angela Merkel, Olaf Scholz und Trump kenne ich  auch nicht, alle leben in Mainz."
test_mails_phone = "Hallo meine email ist test@gmail.com und hi@tester.com. Du kannst mich auch unter +49 172 615 5172 erreichen "
test_IBAN = "Hallo meine IBAN ist DE89 3704 0044 0532 0130 00"




res = anno(test_names, public_figures_pattern, user_patterns, SALT, nlp)

print(res)

PER_USER
@tiktok_nickname
PER
Tino
PER_PUBLIC
Angela Merkel
PER_PUBLIC
Olaf Scholz
PER_PUBLIC
Trump
LOC
Mainz
PerHash_229266925243435989910934436362641138482 und PerHash_154007533813332619190776515691938131007 sind nicht bekannt aber gute Kumpels. Angela Merkel, Olaf Scholz und Trump kenne ich  auch nicht, alle leben in LocHash_69903130440719613191237824624335364436.
